# Chapter 2:

CHANGELOG (commented out code):
- using eval dataset as weave.Dataset

**Comprehensive Evaluation Strategies**

In this chapter, we will evaluate the two main components of a RAG pipeline - retriever and response generator.

Evaluating the retriever can be considered component evaluation. Depending on your RAG pipeline, there can be a few components and for ensuring robustness of your system, it is recommended to come up with evaluation for each components. 


In [1]:
%load_ext autoreload
%autoreload 2


import json
import pathlib

import nest_asyncio
import pandas as pd

import wandb

nest_asyncio.apply()
import asyncio

import weave
from dotenv import load_dotenv

from scripts.utils import display_source

load_dotenv()

True

In the last chapter we used a few features from W&B and initialized (`wandb.init`)a W&B run.

In this chapter we will also use W&B Weave for our evaluation purposes. The `weave.Evaluation` class is a light weight class that can be used to evaluate the performance of a `weave.Model` on a `weave.Dataset`. We will go into more details.

We will initialize a weave client which will track traces and evaluation scores.

In [2]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

wandb.require("core")

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    group="Chapter 2",
)

weave_client = weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ayut (rag-course). Use `wandb login --relogin` to force relogin


weave version 0.50.10 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: ayut.
View Weave data at https://wandb.ai/rag-course/dev/weave


## Building and improving an evaluation dataset

### Collecting data for evaluation

We used our [FAQs](https://docs.wandb.ai/guides/technical-faq) section from the docs website to build our evaluation set. 

The evaluation samples are logged as [`weave.Dataset`](https://wandb.github.io/weave/guides/core-types/datasets/). `weave.Dataset` enable you to collect examples for evaluation and automatically track versions for accurate comparisons. 

Below we will download the latest version locally with a simple API.

In [11]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:9O0EmmPINmYjgbXW3kucVrDxlTUQJQs0fVZYJj2mtOk"
).get()

Iterating through each sample is easy.

We have the question, ground truth answer and ground truth contexts.

In [12]:
dict(eval_dataset.rows[0])

{'question': 'How can I access the run object from the Lightning WandBLogger function?',
 'answer': "In PyTorch Lightning, the `WandbLogger` is used to log metrics, model weights, and other data to Weights & Biases during training. To access the `wandb.Run` object from within a `LightningModule` when using `WandbLogger`, you can use the `Trainer.logger.experiment` attribute. This attribute provides direct access to the underlying `wandb.Run` object, allowing you to interact with the Weights & Biases API directly.\n\nHere's how you can access the `wandb.Run` object using `WandbLogger` in PyTorch Lightning:\n\n```python\nfrom pytorch_lightning import Trainer, LightningModule\nfrom pytorch_lightning.loggers import WandbLogger\n\nclass MyModel(LightningModule):\n    def training_step(self, batch, batch_idx):\n        # Your training logic here\n        loss = ...\n\n        # Log metrics\n        self.log('train_loss', loss)\n\n        # Access the wandb.Run object\n        run = self.trai

In [4]:
# # TODO: Remove this once we more to the final project

# from datetime import datetime

# run = wandb.init(
#     entity="rag-course",
#     project="dev",
#     group="Chapter 2",
# )
# eval_artifact = wandb.Artifact(
#     name="eval_dataset",
#     type="dataset",
#     description="Evaluation dataset for RAG",
#     metadata={
#         "total_samples": {"easy_eval": 20, "hard_eval": 50, "test": 100},
#         "date_collected": datetime.now().strftime("%Y-%m-%d"),
#         "chapter": "Chapter 2",
#     },
# )
# eval_artifact.add_dir("../data/eval")
# run.log_artifact(eval_artifact)
# run.finish()

In [5]:
# TODO: to be removed

# eval_artifact = run.use_artifact(
#     f"{WANDB_ENTITY}/{WANDB_PROJECT}/eval_dataset:latest", type="dataset"
# )
# eval_dir = eval_artifact.download("../data/eval")
# eval_dataset = pd.read_json(
#     f"{eval_dir}/eval_dataset.jsonl", lines=True, orient="records"
# )
# eval_samples = eval_dataset.to_dict(orient="records")
# eval_dataset

### Evaluating the Retriever

This is a search problem, it's easiest to start with tradiaional Information retrieval metrics.


ref: https://weaviate.io/blog/retrieval-evaluation-metrics

**TODO** Add weave model and evals in this section

In [5]:
# Reload the data from Chapter 1
chunked_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest", type="dataset"
)
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

2024/07/19 15:13:53 [DEBUG] GET https://storage.googleapis.com/wandb-production.appspot.com/rag-course/dev/j8uh2i2o/artifact/961260984/wandb_manifest.json?Expires=1721385833&GoogleAccessId=gorilla-files-url-signer-man%40wandb-production.iam.gserviceaccount.com&Signature=nzklkvYCGavFurUUMYIS99h05OXWtrThOuGzfB5BhK6NXYiodXlL5cL5OJQGtZLfRpS48kn3dWn6lRU1gaQ5SjqqOILLQCnr47rEqDRnW4qKozqDxMUK2gD2M9WyF99IWUTuoz4c389YOLS1UxlcPyQMec6wsvc1ltU0cQ5ccjcfctOQgA4xeRAZvE6byjv8i1aiw6GT7kSkWx%2BSNHjLhk0wuh79uOD7LuiTdJkaC%2FX2w2UAfabT5e%2Fc3eV8YEkQctbMw7d%2B8SkileEt2vBA2luDmKdaZuzxa008E0iGH1fl5eDHcFppE7vrpLx8beBI6a6%2FsGfI3bGHRfOIsmWKjw%3D%3D


[{'cleaned_content': 'Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with wandb.init(anonymous="allow") :::info Publishing a paper? Please cite W&B, and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n::: How does someone without an account see results? If someone runs your script and you have to set anonymous="allow":  Auto-create temporary account: W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. Log results quickly: The user can run and re-run the script, and automatically see results show up in the W&B dashboard UI.\nThese unclaimed anonymous runs will be available for 7 days. Claim

We will import the `TFIDFRetriever` which we created in the last chapter and index the chunked data.

In [9]:
from scripts.retriever import TFIDFRetriever

display_source(TFIDFRetriever)

retriever = TFIDFRetriever()
retriever.index_data(chunked_data)

   1 class TFIDFRetriever(weave.Model):                                                                            
   2     vectorizer: TfidfVectorizer = TfidfVectorizer()                                                           
   3     index: list = None                                                                                        
   4     data: list = None                                                                                         
   5                                                                                                               
   6     def index_data(self, data):                                                                               
   7         self.data = data                                                                                      
   8         docs = [doc["cleaned_content"] for doc in data]                                                       
   9         self.index = self.vectorizer.fit_transform(docs)                                                      
  10                                                                                                               
  11     @weave.op()                                                                                               
  12     def search(self, query, k=5):                                                                             
  13         query_vec = self.vectorizer.transform([query])                                                        
  14         cosine_distances = cdist(                                                                             
  15             query_vec.todense(), self.index.todense(), metric="cosine"                                        
  16         )[0]                                                                                                  
  17         top_k_indices = cosine_distances.argsort()[:k]                                                        
  18         output = []                                                                                           
  19         for idx in top_k_indices:                                                                             
  20             output.append(                                                                                    
  21                 {                                                                                             
  22                     "source": self.data[idx]["metadata"]["source"],                                           
  23                     "text": self.data[idx]["cleaned_content"],                                                
  24                     "score": 1 - cosine_distances[idx],                                                       
  25                 }                                                                                             
  26             )                                                                                                 
  27         return output                                                                                         
  28                                                                                                               
  29     @weave.op()[48;2

In [10]:
from scripts.retrieval_metrics import IR_METRICS, LLM_METRICS as RETRIEVAL_LLM_METRICS
from scripts.utils import display_source

for scorer in IR_METRICS:
    display_source(scorer)

   1 @weave.op()                                                                                                   
   2 def compute_hit_rate(                                                                                         
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the hit rate (precision) for a single query.                                                    
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The hit rate (precision).                                                                      
  19                                                                                                               
  20     The hit rate (precision) measures the proportion of retrieved documents that are relevant.                
  21     It is calculated using the following formula:                                                             
  22                                                                                                               
  23     \[ \text{Hit Rate (Precision)} = \frac{\text{Number of Relevant Documents Retrieved}}{\text{Total Number o
  24                                                                                                               
  25     This metric is useful for assessing the accuracy of the retrieval system by determining the relevance of t
  26     ```                                                                                                       
  27     """                                                                                                       
  28     search_results = [doc["source"] for doc in model_output]                                                  
  29     relevant_sources = [                                                                                      
  30         context["source"] for context in contexts if context["relevance"] != 0                                
  31     ]                                                                                                         
  32                                                                                                               
  33     # Calculate the number of relevant documents retrieved                                                    
  34     relevant_retrieved = sum(                                                                                 
  35         1 for source in search_results if source in

   1 @weave.op                                                                                                     
   2 def compute_mrr(                                                                                              
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Mean Reciprocal Rank (MRR) for a single query.                                              
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The MRR score for the given query.                                                             
  19                                                                                                               
  20     MRR measures the rank of the first relevant document in the result list.                                  
  21     It is calculated using the following formula:                                                             
  22                                                                                                               
  23     \[ \text{MRR} = \frac{1}{\text{rank of first relevant document}} \]                                       
  24                                                                                                               
  25     If no relevant document is found, MRR is 0.                                                               
  26                                                                                                               
  27     This metric is useful for evaluating systems where there is typically one relevant document               
  28     and the user is interested in finding that document quickly.                                              
  29     """                                                                                                       
  30     relevant_sources = [                                                                                      
  31         context["source"] for context in contexts if context["relevance"] != 0                                
  32     ]                                                                                                         
  33                                                                                                               
  34     mrr_score = 0                                                                                             
  35     for rank, result in enumerate(model_output, 1):

   1 @weave.op                                                                                                     
   2 def compute_ndcg(                                                                                             
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Normalized Discounted Cumulative Gain (NDCG) for a single query.                            
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The cosine similarity score of the document to the query.                          
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The NDCG score for the given query.                                                            
  19                                                                                                               
  20     NDCG measures the ranking quality of the search results, taking into account the position of relevant docu
  21                                                                                                               
  22     NDCG Formula:                                                                                             
  23     1. Calculate the Discounted Cumulative Gain (DCG):                                                        
  24        \[ \text{DCG}_p = \sum_{i=1}^p \frac{2^{rel_i} - 1}{\log_2(i + 1)} \]                                  
  25        where \( rel_i \) is the relevance score of the document at position \( i \).                          
  26                                                                                                               
  27     2. Calculate the Ideal Discounted Cumulative Gain (IDCG), which is the DCG of the ideal ranking:          
  28        \[ \text{IDCG}_p = \sum_{i=1}^p \frac{2^{rel_i} - 1}{\log_2(i + 1)} \]                                 
  29        where documents are sorted by their relevance scores in descending order.                              
  30                                                                                                               
  31     3. Normalize the DCG by dividing it by the IDCG to get NDCG:                                              
  32        \[ \text{NDCG}_p = \frac{\text{DCG}_p}{\text{IDCG}_p} \]                                               
  33                                                                                                               
  34     This implementation uses continuous relevance scores.                                                     
  35     """                                            

   1 @weave.op()                                                                                                   
   2 def compute_map(                                                                                              
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Mean Average Precision (MAP) for a single query.                                            
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The MAP score for the given query.                                                             
  19                                                                                                               
  20     MAP provides a single-figure measure of quality across recall levels.                                     
  21     For a single query, it's equivalent to the Average Precision (AP).                                        
  22     It's calculated using the following formula:                                                              
  23                                                                                                               
  24     \[ \text{MAP} = \frac{\sum_{k=1}^n P(k) \times \text{rel}(k)}{\text{number of relevant documents}} \]     
  25                                                                                                               
  26     Where:                                                                                                    
  27     - n is the number of retrieved documents                                                                  
  28     - P(k) is the precision at cut-off k in the list                                                          
  29     - rel(k) is an indicator function: 1 if the item at rank k is relevant, 0 otherwise                       
  30     MAP considers both precision and recall, as well as the ranking of relevant documents.                    
  31                                                                                                               
  32     """                                                                                                       
  33     relevant_sources = {                                                                                      
  34         context["source"] for context in contexts if context["relevance"] != 0                                
  35     }                                              

   1 @weave.op()                                                                                                   
   2 def compute_precision(                                                                                        
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Precision for a single query.                                                               
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The Precision score for the given query.                                                       
  19                                                                                                               
  20     Precision measures the proportion of retrieved documents that are relevant.                               
  21     It is calculated using the following formula:                                                             
  22                                                                                                               
  23     \[ \text{Precision} = \frac{\text{Number of Relevant Documents Retrieved}}{\text{Total Number of Documents
  24     """                                                                                                       
  25     relevant_sources = {                                                                                      
  26         context["source"] for context in contexts if context["relevance"] != 0                                
  27     }                                                                                                         
  28     retrieved_sources = {result["source"] for result in model_output}                                         
  29                                                                                                               
  30     relevant_retrieved = relevant_sources & retrieved_sources                                                 
  31                                                                                                               
  32     precision = (                                                                                             
  33         len(relevant_retrieved) / len(retrieved_sources) if retrieved_sources else 0.0                        
  34     )                                                                                                         
  35     return precision                               

   1 @weave.op()                                                                                                   
   2 def compute_recall(                                                                                           
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Recall for a single query.                                                                  
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The Recall score for the given query.                                                          
  19                                                                                                               
  20     Recall measures the proportion of relevant documents that are retrieved.                                  
  21     It is calculated using the following formula:                                                             
  22                                                                                                               
  23     \[ \text{Recall} = \frac{\text{Number of Relevant Documents Retrieved}}{\text{Total Number of Relevant Doc
  24     """                                                                                                       
  25     relevant_sources = {                                                                                      
  26         context["source"] for context in contexts if context["relevance"] != 0                                
  27     }                                                                                                         
  28     retrieved_sources = {result["source"] for result in model_output}                                         
  29                                                                                                               
  30     relevant_retrieved = relevant_sources & retrieved_sources                                                 
  31                                                                                                               
  32     recall = (                                                                                                
  33         len(relevant_retrieved) / len(relevant_sources) if relevant_sources else 0.0                          
  34     )                                                                                                         
  35     return recall                                  

   1 @weave.op()                                                                                                   
   2 def compute_f1_score(                                                                                         
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the F1-Score for a single query.                                                                
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The F1-Score for the given query.                                                              
  19                                                                                                               
  20     F1-Score is the harmonic mean of Precision and Recall.                                                    
  21     It is calculated using the following formula:                                                             
  22                                                                                                               
  23     \[ \text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall
  24     """                                                                                                       
  25     precision = compute_precision(model_output, contexts)                                                     
  26     recall = compute_recall(model_output, contexts)                                                           
  27                                                                                                               
  28     if precision + recall == 0:                                                                               
  29         return 0.0                                                                                            
  30                                                                                                               
  31     f1_score = 2 * (precision * recall) / (precision + recall)                                                
  32     return f1_score                                                                                           
  33                                                                                                               

#### Evaluating retrieval on other metrics

In [13]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k":5}
)
retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))

Evaluated 1 of 50 examples

Evaluated 2 of 50 examples

Evaluated 3 of 50 examples

Evaluated 4 of 50 examples

Evaluated 5 of 50 examples

...

Evaluated 48 of 50 examples

Evaluated 49 of 50 examples

Evaluated 50 of 50 examples

Evaluation summary
{
    'compute_hit_rate': {'mean': 0.36800000000000005},
    'compute_mrr': {'mean': 0.6376666666666666},
    'compute_ndcg': {'mean': 0.84},
    'compute_map': {'mean': 0.3721126984126984},
    'compute_precision': {'mean': 0.35200000000000004},
    'compute_recall': {'mean': 0.3675714285714286},
    'compute_f1_score': {'mean': 0.35215584415584417},
    'model_latency': {'mean': 0.06725754261016846}
}

🍩 https://wandb.ai/rag-course/dev/r/call/abeb2388-df96-47ae-b11b-80bb8748c52d


### Using an LLM as a Retrieval Judge

**ref: https://arxiv.org/pdf/2406.06519**

How do we evaluate if we don't have any ground truth? 

We can use a powerful LLM as a judge to evaluate the retriever. 


In [14]:
for metric in RETRIEVAL_LLM_METRICS:
    display_source(metric)

   1 @weave.op()                                                                                                   
   2 async def llm_retrieval_scorer(                                                                               
   3     model_output: Dict[str, Any], question: str                                                               
   4 ) -> Dict[str, float]:                                                                                        
   5     scores = await evaluate_retrieval_with_llm(question, model_output)                                        
   6     relevance_scores = [item["relevance"] for item in scores["final_scores"]]                                 
   7     mean_relevance = sum(relevance_scores) / len(model_output)                                                
   8     rank_score = compute_rank_score(relevance_scores)                                                         
   9     return {"relevance": mean_relevance, "relevance_rank_score": rank_score}                                  
  10                                                                                                               

In [15]:
retrieval_evaluation = weave.Evaluation(
    name="LLM_Judge_Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=RETRIEVAL_LLM_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k":5}
)
retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))

Evaluated 1 of 50 examples

Evaluated 2 of 50 examples

Evaluated 3 of 50 examples

Evaluated 4 of 50 examples

Evaluated 5 of 50 examples

...

Evaluated 48 of 50 examples

Evaluated 49 of 50 examples

Evaluated 50 of 50 examples

Evaluation summary
{
    'llm_retrieval_scorer': {'relevance': {'mean': 0.504}, 'relevance_rank_score': {'mean': 0.28333333333333327}},
    'model_latency': {'mean': 1.7746700525283814}
}

🍩 https://wandb.ai/rag-course/dev/r/call/51239910-956f-4e7c-840a-8debb0a03f8f


## Evaluating the Response

In [16]:
from scripts.rag_pipeline import SimpleRAGPipeline
from scripts.response_generator import SimpleResponseGenerator

INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
rag_pipeline = SimpleRAGPipeline(retriever=retriever, response_generator=response_generator, top_k=5)

In [17]:
from scripts.response_metrics import NLP_METRICS, LLM_METRICS as RESPONSE_LLM_METRICS
for scorer in NLP_METRICS:
    display_source(scorer)

   1 @weave.op()                                                                                                   
   2 def compute_diff(model_output: str, answer: str) -> float:                                                    
   3     """                                                                                                       
   4     Compute the similarity ratio between the normalized model output and the expected answer.                 
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The output generated by the model.                                                
   8         answer (str): The expected answer.                                                                    
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The similarity ratio between the normalized model output and the expected answer.              
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     return difflib.SequenceMatcher(None, norm_output, norm_answer).ratio()                                    
  16                                                                                                               

   1 @weave.op()                                                                                                   
   2 def compute_levenshtein(model_output: str, answer: str) -> float:                                             
   3     """                                                                                                       
   4     Compute the Levenshtein ratio between the normalized model output and the answer.                         
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The output generated by the model.                                                
   8         answer (str): The expected answer.                                                                    
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The Levenshtein ratio between the normalized model output and the answer.                      
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     return Levenshtein.ratio(norm_output, norm_answer)                                                        
  16                                                                                                               

   1 @weave.op()                                                                                                   
   2 def compute_rouge(model_output: str, answer: str) -> float:                                                   
   3     """                                                                                                       
   4     Compute the ROUGE-L F1 score between the normalized model output and the reference answer.                
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The model's generated output.                                                     
   8         answer (str): The reference answer.                                                                   
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The ROUGE-L F1 score.                                                                          
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     rouge = Rouge(metrics=["rouge-l"], stats="f")                                                             
  16     scores = rouge.get_scores(norm_output, norm_answer)                                                       
  17     return scores[0]["rouge-l"]["f"]                                                                          
  18                                                                                                               

   1 @weave.op()                                                                                                   
   2 def compute_bleu(model_output: str, answer: str) -> float:                                                    
   3     """                                                                                                       
   4     Compute the BLEU score between the normalized model output and the reference answer.                      
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The generated output from the model.                                              
   8         answer (str): The reference answer.                                                                   
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The BLEU score between the normalized model output and the reference answer.                   
  12     """                                                                                                       
  13     chencherry = SmoothingFunction()                                                                          
  14     smoothing_function = chencherry.method2                                                                   
  15                                                                                                               
  16     norm_output = normalize_text(model_output)                                                                
  17     norm_answer = normalize_text(answer)                                                                      
  18     reference = word_tokenize(norm_answer)                                                                    
  19     candidate = word_tokenize(norm_output)                                                                    
  20     score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)                      
  21     return score                                                                                              
  22                                                                                                               

   1 @weave.op                                                                                                     
   2 def compute_meteor(model_output: str, answer: str) -> float:                                                  
   3     """                                                                                                       
   4     Compute the METEOR score between the normalized model output and the reference answer.                    
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The model's generated output.                                                     
   8         answer (str): The reference answer.                                                                   
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The METEOR score rounded to 4 decimal places.                                                  
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     reference = word_tokenize(norm_answer)                                                                    
  16     candidate = word_tokenize(norm_output)                                                                    
  17     meteor_score = round(meteor([candidate], reference), 4)                                                   
  18     return meteor_score                                                                                       
  19                                                                                                               

In [19]:
response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=NLP_METRICS[:-1],
    preprocess_model_input=lambda x: {"query": x["question"]})
response_scores = asyncio.run(response_evaluations.evaluate(rag_pipeline))

Evaluated 1 of 50 examples

Evaluated 2 of 50 examples

Evaluated 3 of 50 examples

Evaluated 4 of 50 examples

Evaluated 5 of 50 examples

...

Evaluated 48 of 50 examples

Evaluated 49 of 50 examples

Evaluated 50 of 50 examples

Evaluation summary
{
    'compute_diff': {'mean': 0.02537405600482},
    'compute_levenshtein': {'mean': 0.33773542778949595},
    'compute_rouge': {'mean': 0.19529523604538812},
    'compute_bleu': {'mean': 0.0223811944166886},
    'model_latency': {'mean': 5.233205227851868}
}

🍩 https://wandb.ai/rag-course/dev/r/call/0a699274-7490-4956-a0ec-0ee7c468ac94


### Using an LLM as a Response Judge

Some metrics cannot be defined objectively and are particularly useful for more subjective or complex criteria.
We care about correctness, faithfulness, and relevance.

- **Answer Correctness** - Is the generated answer correct compared to the reference and thoroughly answers the user's query?
- **Answer Relevancy** - Is the generated answer relevant and comprehensive?
- **Answer Factfulness** - Is the generated answer factually consistent with the context document?


In [20]:
for metric in RESPONSE_LLM_METRICS:
    display_source(metric)

   1 @weave.op()                                                                                                   
   2 async def llm_response_scorer(                                                                                
   3     model_output: str, question: str, answer: str                                                             
   4 ) -> Dict[str, Any]:                                                                                          
   5     evaluation_result = await evaluate_correctness_using_llm_judge(                                           
   6         question, answer, model_output                                                                        
   7     )                                                                                                         
   8                                                                                                               
   9     return {                                                                                                  
  10         "score": evaluation_result["final_score"],                                                            
  11         "correct": evaluation_result["decision"] == "correct",                                                
  12     }                                                                                                         
  13                                                                                                               

In [21]:
correctness_evaluations = weave.Evaluation(
    name="Correctness_Evaluation",
    dataset=eval_dataset,
    scorers=RESPONSE_LLM_METRICS, 
    preprocess_model_input=lambda x: {"query": x["question"]})
response_scores = asyncio.run(correctness_evaluations.evaluate(rag_pipeline))

Evaluated 1 of 50 examples

Evaluated 2 of 50 examples

Evaluated 3 of 50 examples

Evaluated 4 of 50 examples

Evaluated 5 of 50 examples

...

Evaluated 48 of 50 examples

Evaluated 49 of 50 examples

Evaluated 50 of 50 examples

Evaluation summary
{
    'llm_response_scorer': {'score': {'mean': 0.68}, 'correct': {'true_count': 6, 'true_fraction': 0.12}},
    'model_latency': {'mean': 6.152640972137451}
}

🍩 https://wandb.ai/rag-course/dev/r/call/7f34a54b-af15-468f-bd0a-5330e61e749c


## Exercise

1. Implement the `Relevance` and `Faithfulness` evaluators and evaluate the pipeline on all the dimensions.
2. Generate and share a W&B report with the following sections in the form of tables and charts:
    
    - Summary of the evaluation
    - Retreival Evaluations
        - IR Metrics
        - LLM As a Retrieval Judge Metric
    - Response Evalations
        - Traditional NLP Metrics
        - LLM Judgement Metrics
    - Overall Evalations
    - Conclusion
